In [16]:
import os
import csv
import pandas as pd
import numpy as np
import boto3
from IPython.display import clear_output
import ast
import pymongo as pm
from bson.objectid import ObjectId

## NEED TO RUN NOTEBOOK AS PYTHON3 ## 
## Otherwise can't connect using boto3 due to SSL issue with python2; installing packages using pip3 and python3 
## kernel solved the issue

In [17]:
version = 'cdm_run_v7'
csv_path = '/Users/brialong/Documents/GitHub/kiddraw/data/drawings/' + version + '/filtered_dataset_descriptives/' 
file_name = version + 'objects_filtered_2020-09-15.csv'
full_path = os.path.join(csv_path, file_name)
data = pd.read_csv(full_path)
data

,Unnamed: 0,fileNameShort,session_id,trial_num,category,age,submit_time,submit_date,num_strokes,draw_duration_old,draw_duration_new,trial_duration,mean_intensity,bounding_box,filename,version,fullPath,age_numeric,fileNameShortNew
0,1,a bee_sketch_age10_cdm_run_v71571526669020.png,cdm_run_v71571526669020,8,a bee,age10,1571526876038,2019-10-19 23:14:36,9,11.136,0,14.523,0.028181,"(57, 163, 22, 150)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,10,bee_sketch_age10_cdm_run_v71571526669020.png
1,2,a bee_sketch_age10_cdm_run_v71571610102326.png,cdm_run_v71571610102326,6,a bee,age10,1571610254339,2019-10-20 22:24:14,11,11.876,0,14.910,0.071568,"(21, 145, 22, 187)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,10,bee_sketch_age10_cdm_run_v71571610102326.png
2,3,a bee_sketch_age10_cdm_run_v71571610980974.png,cdm_run_v71571610980974,10,a bee,age10,1571611186483,2019-10-20 22:39:46,9,7.563,0,9.990,0.035635,"(25, 121, 49, 211)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,10,bee_sketch_age10_cdm_run_v71571610980974.png
3,4,a bee_sketch_age10_cdm_run_v71572124012267.png,cdm_run_v71572124012267,11,a bee,age10,1572124312613,2019-10-26 21:11:52,20,27.762,0,32.490,0.063237,"(6, 162, 28, 190)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,10,bee_sketch_age10_cdm_run_v71572124012267.png
4,5,a bee_sketch_age10_cdm_run_v71572205431405.png,cdm_run_v71572205431405,6,a bee,age10,1572205523288,2019-10-27 19:45:23,7,2.785,0,4.938,0.026547,"(30, 216, 94, 156)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,10,bee_sketch_age10_cdm_run_v71572205431405.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,7329,an octopus_sketch_age9_cdm_run_v71576700191030...,cdm_run_v71576700191030,6,an octopus,age9,1576700357845,2019-12-18 20:19:17,15,18.393,0,22.190,0.095663,"(54, 208, 0, 214)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,9,octopus_sketch_age9_cdm_run_v71576700191030.png
7329,7330,an octopus_sketch_age9_cdm_run_v71576884416164...,cdm_run_v71576884416164,8,an octopus,age9,1576884708243,2019-12-20 23:31:48,16,12.597,0,18.235,0.070950,"(0, 223, 0, 223)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,9,octopus_sketch_age9_cdm_run_v71576884416164.png
7330,7331,an octopus_sketch_age9_cdm_run_v71576958347940...,cdm_run_v71576958347940,5,an octopus,age9,1576958488834,2019-12-21 20:01:28,4,2.157,0,14.225,0.053651,"(39, 192, 21, 195)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,9,octopus_sketch_age9_cdm_run_v71576958347940.png
7331,7332,an octopus_sketch_age9_cdm_run_v71577044793131...,cdm_run_v71577044793131,7,an octopus,age9,1577044943507,2019-12-22 20:02:23,7,1.221,0,8.316,0.003268,"(158, 195, 152, 189)",/Users/brialong/Documents/GitHub/kiddraw/analy...,cdm_run_v7,/Users/brialong/Documents/GitHub/kiddraw/data/...,9,octopus_sketch_age9_cdm_run_v71577044793131.png


In [18]:


data = data[['session_id', 'category', 'age','fileNameShortNew']]
data = data[~data['category'].isin(['something you love'])]
data.reset_index().drop(['index'], axis=1)
data['filename'] = data['fileNameShortNew']
data['url'] = data['filename'].apply(lambda x: 'https://kiddraw.s3.amazonaws.com/' + x)
data = data.rename(columns={'category':'class'})

# add a shuffle id and a games variable to indicate the number of times the current image is labelled
data_range = np.arange(data.shape[0])
np.random.shuffle(data_range)
data['shuffler_ind'] = data_range
data['games'] = '[]'
data['games'] = data['games'].apply(lambda x: ast.literal_eval(x))



In [19]:
np.size(np.unique(data.filename))

7333

#### Upload images to s3

In [20]:
path_to_png = '/Users/brialong/Documents/GitHub/kiddraw/data/drawings/cdm_run_v7/filtered_dataset'


In [21]:
np.size(os.listdir(path_to_png))

7333

In [22]:
runThis = 1
bucket_name = 'kiddraw'
if runThis:
    conn = boto3.resource('s3')
    b = conn.Bucket(bucket_name) 
b


s3.Bucket(name='kiddraw')

In [23]:
runThis = 1
path_to_png = '/Users/brialong/Documents/GitHub/kiddraw/data/drawings/cdm_run_v7/filtered_dataset'
bucket_name = 'kiddraw'
if runThis:
    conn = boto3.resource('s3')
    b = conn.Bucket(bucket_name) 
    for ind,im in enumerate(os.listdir(path_to_png)):
        if im[-3:]=='png':
            print (ind, im)
            img_obj = b.Object(im)
            img_obj.upload_file(os.path.join(path_to_png, im))
            img_obj.Acl().put(ACL='public-read')
            clear_output(wait=True)  
# print 'Done!'

7332 clock_sketch_age6_cdm_run_v71575572118528.png


In [13]:
# for key in b.objects.all():
#     print key.key


#### Convert the csv file to json and upload to mongoDB

In [24]:
# convert the csv file to json
import json
datajs = data.to_json(orient='records')
datajs = json.loads(datajs)

In [25]:
datajs

[{'session_id': 'cdm_run_v71571526669020',
  'class': 'a bee',
  'age': 'age10',
  'fileNameShortNew': 'bee_sketch_age10_cdm_run_v71571526669020.png',
  'filename': 'bee_sketch_age10_cdm_run_v71571526669020.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bee_sketch_age10_cdm_run_v71571526669020.png',
  'shuffler_ind': 1822,
  'games': []},
 {'session_id': 'cdm_run_v71571610102326',
  'class': 'a bee',
  'age': 'age10',
  'fileNameShortNew': 'bee_sketch_age10_cdm_run_v71571610102326.png',
  'filename': 'bee_sketch_age10_cdm_run_v71571610102326.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bee_sketch_age10_cdm_run_v71571610102326.png',
  'shuffler_ind': 1236,
  'games': []},
 {'session_id': 'cdm_run_v71571610980974',
  'class': 'a bee',
  'age': 'age10',
  'fileNameShortNew': 'bee_sketch_age10_cdm_run_v71571610980974.png',
  'filename': 'bee_sketch_age10_cdm_run_v71571610980974.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bee_sketch_age10_cdm_run_v71571610980974.png',
  'shuffler

In [30]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the stanford-cogsci.org user
pswd = auth.values[0][0]

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
# coll = db['check_invalid_cdm_run_v5_production'] 
coll = db['check_invalid_cdm_run_v7_production'] 

In [31]:
coll.count()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


7333

In [32]:
# # drop past data
# reallyRun = 1
# if reallyRun:
#     coll.drop()


In [33]:
# upload data to the development collection
reallyRun = 1
if reallyRun:
    for i,_z in enumerate(datajs):
        coll.insert_one(_z)
        print('Inserted {} records.'.format(i))
        clear_output(wait=True)
else:
    print('Did not insert any new data.')

DuplicateKeyError: E11000 duplicate key error collection: kiddraw.check_invalid_cdm_run_v7_production index: _id_ dup key: { : ObjectId('5f625e269d2bb9d2b91af2b4') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: kiddraw.check_invalid_cdm_run_v7_production index: _id_ dup key: { : ObjectId('5f625e269d2bb9d2b91af2b4') }"}

In [15]:
## check how many records have been retrieved on the mturk study
# print 'Currently there are {} records in the collection'.format(coll.count())
#{"games": {"$size": 1}}
# {"_id": ObjectId("5e6a7e7d1943e62410f1ebb3")}
a = coll.find()
numGames = []
for rec in a:
    if len(rec['games']) > 0:
        numGames.append(len(rec['games'])) 
print (len(numGames))

0
